In [ ]:
import json
# Load JSON data
with open("../data/home0001qa.json", "r") as file:
    qa_data = json.load(file)

Because we have a question / answer dataset with relatively short answers, we don't need to use a splitter

In [ ]:
from langchain.schema import Document
# Prepare documents for LangChain
documents = [
    Document(page_content=item["answer"], metadata={"question": item["question"]})
    for item in qa_data
]

documents[:5]

In [ ]:
# turn into function
def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content=item["answer"],
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

print(prepare_qa_documents("../data/home0001qa.json")[:5])

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [35]:
def setup_rag(documents):
    
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents, embeddings)
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following similar Q&A pairs to help answer the question. 
        If the context is relevant, use it to answer. If not, say you don't have enough information.
        
        Context Q&A pairs:
        {context}
        """),
        ("human", "{question}")
    ])
    
    chain = (
        {"context": vectorstore.as_retriever(search_type="similarity", k=3), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [ ]:
docs = prepare_qa_documents('../data/home0001qa.json')
chain = setup_rag(docs)

In [ ]:
print(chain.invoke("Do i own my 0001 home outright?"))

In [ ]:
for chunk in chain.stream("What is Home0001?"):
    print(chunk, end="", flush=True)